# Rellenar días sin registro para la variable
Se rellena el valor del día con la mediana de los valores que se tuvieron para el mismo día y mes en todo el rango y sólo si existen mínimo 10 años

## Cargar librerias

In [1]:
import os
import glob
import pandas as pd
import numpy as np
from datetime import datetime
import time

## Asignar parámetros para el script

In [10]:
variable_path_base = r'../data/variables'
variable_files_pattern = f'var-dia-TMN_CON-raw.csv'
variable_file_output = r'var-dia-{etiqueta}-fill.csv'

column_to_fill = 'Valor'
column_filled = 'ValorFilled'

aggregate_func = 'median'
YEARS_MIN_TO_FILL = 10

## Determinar archivos a procesar

In [11]:
variable_path_base = variable_path_base.split('/')
variable_files = glob.glob(os.path.join(*variable_path_base, variable_files_pattern))
print('Archivos a cargar', *variable_files, sep = '\n')

Archivos a cargar
..\data\variables\var-dia-TMN_CON-raw.csv


## Leer variable

In [8]:
def read_variable(f):
    dtypes = { 'CodigoEstacion':'category','Etiqueta':'category'}
    dateparse = lambda x: datetime.strptime(x, '%Y-%m-%d %H:%M')
    
    df = pd.read_csv(f, dtype = dtypes, parse_dates = ['Fecha']) #, nrows = 10000
    return df

## Incluir los días faltantes dejándo Vacío el valor

In [5]:
def fill_missing_days(df):
    #https://stackoverflow.com/a/44978400/1828356
    dates = pd.date_range(start = df.Fecha.min(), end = df.Fecha.max())
    stations = df.CodigoEstacion.unique()
    idx_names = ['CodigoEstacion', 'Fecha']
    idx = pd.MultiIndex.from_product((stations, dates), names = idx_names)
    df = df.set_index(idx_names).reindex(idx)
    df.Etiqueta = etiqueta
    return df

## Rellenar el valor
### Mejoras
* Sólo procesar los días que tuvieron algún vacío
* Sólo rellenar los días en el rango 2000 a 2019

In [6]:
def fill_missing_values(df):
    res = []
    for name, group in df.groupby(['CodigoEstacion']):
        std = group[column_to_fill].std()
        median = group[column_to_fill].median()
        
        group.loc[:,'DevMedian'] = round(group[column_to_fill].apply(lambda x: (x - median) / std), 1)
        
        group.loc[:, column_filled] = (group.groupby([group.index.get_level_values('Fecha').strftime('%m-%d')])[column_to_fill]
                                          #.transform(lambda x: x.fillna(np.NAN if x.count() >= YEARS_MIN_TO_FILL else x.agg(aggregate_func) ))
                                           .apply(lambda x: x.fillna(np.NAN if x.count() < YEARS_MIN_TO_FILL else x.agg(aggregate_func)))
                                      )
        
        res.append(group)

    df = pd.concat(res)
    return df

## Procesar los archivos

In [12]:
%%time
for f in variable_files:
#f = variable_files[1]
    
    start_time = time.time()

    print(f'Archivo: {f}')
    print(f'Leyendo h:{time.asctime()}...')

    df = read_variable(f)

    etiqueta = df.iat[0,1]

    print(f'**** r:{df.shape} h:{time.asctime()} t:{(time.time() - start_time) / 60} ****')
    #df.head()

    print(f'Llenando días faltantes...')
    df = fill_missing_days(df)

    #idx_names = ['CodigoEstacion', 'Fecha']
    #df = df.set_index(idx_names)

    print(f'**** r:{df.shape} h:{time.asctime()} t:{(time.time() - start_time) / 60} ****')

    print(f'Llenando valores faltantes...')
    df = fill_missing_values(df)
    print(f'**** r:{df.shape} h:{time.asctime()} t:{(time.time() - start_time) / 60} ****')

    print('Completos: ', df[(~df[column_to_fill].isna())].shape)
    print('Rellenados: ', df[(df[column_to_fill].isna()) & (~df[column_filled].isna())].shape)
    print('Sin poder rellenar', df[(df[column_to_fill].isna()) & (df[column_filled].isna())].shape)
    
    #df['DevMedian'] = df['DevMedian'].map(lambda x: '%.1f' % x)

    p = os.path.join(*variable_path_base, 
                     variable_file_output.format(etiqueta = etiqueta, aggregate_func = aggregate_func, YEARS_MIN_TO_FILL = YEARS_MIN_TO_FILL))
    print(f'Almacenando en {p}...')
    df.to_csv(p)
    print(f'**** h:{time.asctime()} t:{(time.time() - start_time) / 60} ****')

Archivo: ..\data\variables\var-dia-TMN_CON-raw.csv
Leyendo h:Thu Jul  2 18:57:23 2020...
**** r:(2705149, 4) h:Thu Jul  2 18:57:27 2020 t:0.070058806737264 ****
Llenando días faltantes...
**** r:(4319088, 2) h:Thu Jul  2 18:58:34 2020 t:1.1907422383626303 ****
Llenando valores faltantes...
**** r:(4319088, 4) h:Thu Jul  2 19:04:51 2020 t:7.469285571575165 ****
Completos:  (2705149, 4)
Rellenados:  (891418, 4)
Sin poder rellenar (722521, 4)
Almacenando en ..\data\variables\var-dia-TMN_CON-fill.csv...
**** h:Thu Jul  2 19:05:29 2020 t:8.104868892828623 ****
Wall time: 8min 6s
